In [ ]:
import pandas as pd
import re
import statsapi
from datetime import datetime

import constants
import utils

## TODO: Take the sheet directly from Google Sheets instead of CSV

1. Get list of all players from fantasypros
2. Get list of owned players from Google Sheets
3. Clean owned players data
4. Switch all players naming (Ronald Acuna Jr.) to PL format (RAcuñaJr-OFAtl)
5. Delta

In [ ]:
pl_export = utils.load_dataframe(constants.OWNED_PLAYERS_FILENAME)

In [ ]:
owned_players = utils.clean_pl_export(pl_export)

In [ ]:
single_teams = []
for column in owned_players.columns:
    if column == 'Team Position':
        continue
    single_team = owned_players[['Team Position', column]].copy()
    single_team.columns = ['Team Position', 'Player']
    single_team['PL Owner'] = column
    single_teams.append(single_team)
len(single_teams)

In [ ]:
owned_players_full = pd.concat(single_teams)
# Remove empty bench spots
owned_players_full = owned_players_full.loc[~pd.isna(owned_players_full['Player'])]
owned_players_full

In [ ]:
owned_players_full[['PL Name', 'Pos Team']] = owned_players_full['Player'].str.rsplit('-', n=1, expand=True)
owned_players_full['Player Positions'] = owned_players_full['Pos Team'].str.strip().str[:-3]
owned_players_full['Team'] = owned_players_full['Pos Team'].str.strip().str[-3:].str.upper()
owned_players_full

In [ ]:
def get_player_id_from_pl_name(pl_name, pl_team, season=datetime.now().year, sport_id=constants.SPORT_IDS['MLB']):
    player_id = None
    search_value = None

    # Split the PL-formatted name (JTRealmuto) on capital letters. Take the last value (name) unless special handling is required.
    split_name = re.findall('[A-Z][^A-Z]*', pl_name)
    first_initial = split_name[0][0]
    last_name = split_name[-1]

    # Handle special cases...
    if pl_name in constants.PLAYER_LOOKUP_VALUES.keys():
        search_value = constants.PLAYER_LOOKUP_VALUES[pl_name]
    elif last_name.upper() == 'JR':
        search_value = split_name[-2]
    else:
        search_value = last_name

    # Lookup player in statsapi
    lookup_results = statsapi.lookup_player(search_value, season=season, sportId=sport_id)
    print(f'Got {len(lookup_results)} results from the API for player {pl_name}, team {pl_team}, search value {search_value}, season {season}')

    if pl_name in constants.PLAYER_LOOKUP_VALUES.keys():
        player_id = next((player for player in lookup_results), {}).get('id')
        print(f'Handled special case for player {pl_name}, using search value {search_value}')
    else:
        for player in lookup_results:
            team_id = utils.get_team_id(pl_team)
            # Only check the team ID if it's not a minor league team
            if first_initial == player['fullName'][0] and (team_id == player['currentTeam']['id'] or sport_id != constants.SPORT_IDS['MLB']):
                player_id = player['id']
                break

    if player_id is None:
        # Catch for players who haven't played yet in 2024
        if season >= (datetime.now().year - 2):
            # Catch for players who have never played in the MLB
            if sport_id == constants.SPORT_IDS['MLB']:
                return get_player_id_from_pl_name(pl_name, pl_team, season=season, sport_id=constants.SPORT_IDS['AAA'])
            else:
                return get_player_id_from_pl_name(pl_name, pl_team, season=(season - 1))
        raise ValueError(f'Could not match PL player {pl_name} to an ID, lookup results: {lookup_results}')

    return player_id

In [ ]:
owned_players_full['Player ID'] = owned_players_full.apply(lambda row: get_player_id_from_pl_name(row['PL Name'], row['Team']), axis=1)
owned_players_full

In [ ]:
utils.save_dataframe(owned_players_full, constants.OWNED_PLAYERS_IDS_FILENAME)